# An Interactive Introduction to CFD with ANSYS Fluent

**Objective:** This notebook will guide you through the fundamentals of Computational Fluid Dynamics (CFD) and provide a hands-on experience using ANSYS Fluent to simulate a classic fluid dynamics problem. 
I will give a short lecture on the underlying physics, but again, you only need to understand just enough to get started. 

**Target Audience:** This lesson is designed for students who are new to CFD and ANSYS Fluent.

### Prerequisites

Before you begin, please ensure you have the following:

1.  **ANSYS Fluent:** Version 2022 R2 or newer installed on your computer.
2.  **Python:** Python 3.7 or later. The [Anaconda Distribution](https://www.anaconda.com/products/distribution) is highly recommended as it simplifies package management.
3.  **Required Python Libraries:** You will need `ansys-fluent-core`, `pyvista`, and `matplotlib`. If you don't have them, you can install them by running the following command in your terminal or command prompt:

    ```bash
    pip install ansys-fluent-core pyvista matplotlib
    ```
4. **Read** the excerpt on basic fluid dynamics

I will check that you have everything before we begin do not worry, this section is mostly for me to remember what I need to get you

--- 
## Part 1: The Physics Behind the Simulation - What is CFD?

Computational Fluid Dynamics (CFD) is a branch of fluid mechanics that uses numerical analysis and data structures to analyze and solve problems involving fluid flows. In short, we use computers to solve the governing equations of fluid motion for a specific problem.

### The Governing Equations: Navier-Stokes

At the heart of most CFD problems are the **Navier-Stokes equations**. These are a set of partial differential equations describing the motion of viscous fluid substances. They are built upon three fundamental principles of conservation:

*   **Conservation of Mass:** Mass is never created or destroyed. For a fluid, this means that if flow enters a defined volume, it must also exit it (unless the density changes).
*   **Conservation of Momentum:** This is Newton's Second Law (F=ma) for a fluid. It states that the rate of change of momentum of a fluid particle is equal to the sum of the forces acting on it (like pressure and friction).
*   **Conservation of Energy:** Energy is never created or destroyed. This principle accounts for the transfer of heat and the work done on the fluid.

Solving these equations by hand is only possible for very simple cases. CFD software like ANSYS Fluent solves them numerically over thousands or millions of small cells that make up the geometry of our problem.

--- 
## Part 2: The CFD Workflow

A typical CFD simulation follows a standard workflow. We will automate this entire process using Python within this notebook.

1.  **Preprocessing:**
    *   **Geometry:** Defining the physical shape of the problem (e.g., a pipe, a wing, a heat exchanger).
    *   **Meshing:** Dividing the geometry into thousands or millions of smaller elements called "cells". The governing equations are solved on these cells.

2.  **Solving:**
    *   **Setup:** Defining the physics of the problem (e.g., is the flow turbulent or smooth?), material properties (e.g., air, water), and boundary conditions (e.g., inlet velocity, outlet pressure).
    *   **Solution:** The solver software iteratively calculates the solution for variables like velocity and pressure at each cell in the mesh until the solution is "converged" (meaning the solution is no longer changing significantly).

3.  **Post-processing:**
    *   **Analysis & Visualization:** Examining the results through plots, contours, and vectors to understand the fluid's behavior and extract meaningful data.

--- 
## Part 3: Interactive Simulation - Mixing Elbow
Let's put theory into practice, We will simulate a common engineering problem: a mixing elbow! In this problem, a cold fluid enters from one inlet and a hot fluid enters from another. They mix in the elbow and exit through a single outlet. We will find the resulting temperature and velocity fields. It is one of THE most common engineering problems in all fields, especially aerospace and chemical!

### Step 1: Launch Fluent and Import the Mesh
First, we import the necessary libraries. `pyfluent` allows us to control Fluent from Python. Then, we will launch a Fluent session and load a pre-made mesh file.

In [ ]:
# Import necessary libraries
import ansys.fluent.core as pyfluent
from ansys.fluent.core import examples

# Launch a Fluent session in the background.
# This may take a few moments to start.
# To see the Fluent application window, you can change show_gui to True.
session = pyfluent.launch_fluent(show_gui=False, precision="double", processor_count=2)

# Download an example mesh file for a 3D mixing elbow
import_filename = examples.download_file("mixing_elbow.msh.h5", "pyfluent/mixing_elbow")

# Read the mesh file into Fluent
session.file.read_mesh(file_name=import_filename)

print("Fluent session launched and mesh has been loaded.")

### Step 2: Setting up the Physics
Now we need to tell Fluent about the physics. Our flow will be turbulent, so we will select an appropriate turbulence model. We also need to turn on the energy equation so we can simulate temperature.

In [ ]:
# Access the setup menu for easy access to settings
setup = session.setup

# Set the turbulence model. The k-epsilon model is a common choice.
setup.models.viscous.model = "k-epsilon"

# Turn on the energy equation to solve for temperature
setup.models.energy.enabled = True

print("Models set: k-epsilon turbulence and energy equation enabled.")

#### **Practice Question:**
*What is the key difference between a laminar (smooth) flow and a turbulent (chaotic) flow? Why is it important to choose the correct model for a simulation?*

### Step 3: Defining Boundary Conditions

Boundary conditions (BCs) tell the solver how the fluid behaves at the edges (boundaries) of our domain. We need to set the velocity and temperature at our two inlets and the pressure at our outlet.

In [ ]:
# Access the boundary conditions menu
boundary_conditions = setup.boundary_conditions

# Set the cold inlet velocity (0.4 m/s) and temperature (293.15 K)
boundary_conditions.velocity_inlet["cold-inlet"].vmag = 0.4
boundary_conditions.velocity_inlet["cold-inlet"].t = 293.15

# Set the hot inlet velocity (1.2 m/s) and temperature (313.15 K)
boundary_conditions.velocity_inlet["hot-inlet"].vmag = 1.2
boundary_conditions.velocity_inlet["hot-inlet"].t = 313.15

# Set the outlet pressure to be atmospheric (0 Gauge Pressure)
boundary_conditions.pressure_outlet["outlet"].p_gauge = 0

print("Boundary conditions have been set.")
print(f"Cold Inlet: {boundary_conditions.velocity_inlet['cold-inlet'].vmag()} m/s, {boundary_conditions.velocity_inlet['cold-inlet'].t()} K")
print(f"Hot Inlet: {boundary_conditions.velocity_inlet['hot-inlet'].vmag()} m/s, {boundary_conditions.velocity_inlet['hot-inlet'].t()} K")

#### **Practice Question:**
*The walls of the elbow have a "no-slip" condition by default. What does this mean in terms of the fluid's velocity right at the wall surface?*

### Step 4: Solving the Equations

We are now ready to run the simulation! We will first initialize the flow field, which provides the solver with an initial guess. Then, we run the calculation for a set number of iterations. The solver will stop when the solution is stable, or "converged".

In [ ]:
# Initialize the flow field using hybrid initialization
session.solution.initialization.hybrid_initialize()

print("Solution initialized. Starting calculation...")

# Run the calculation for 150 iterations
session.solution.run_calculation.iterate(iter_count=150)

print("Calculation complete!")

--- 
## Part 4: Post-processing and Analyzing the Results

The simulation is finished! Now comes the fun part: visualizing the results to understand what happened inside the elbow.

We will create contour plots to see the velocity and temperature distribution on a center plane of the elbow.

In [ ]:
# Access the post-processing and graphics tools
results = session.results

# Create a contour plot for temperature
results.graphics.contour["temperature-contour"] = {}
temp_contour = results.graphics.contour["temperature-contour"]
temp_contour.field = "temperature"
temp_contour.surfaces_list = ["symmetry-xyplane"]

# Display the temperature contour plot
# This will save an image file and then display it in the notebook.
temp_contour.display(picture_options={"x_resolution": 1024, "y_resolution": 768})
print("Temperature contour plot generated and displayed.")

In [ ]:
# Create a contour plot for velocity
results.graphics.contour["velocity-contour"] = {}
vel_contour = results.graphics.contour["velocity-contour"]
vel_contour.field = "velocity-magnitude"
vel_contour.surfaces_list = ["symmetry-xyplane"]

# Display the velocity contour plot
vel_contour.display(picture_options={"x_resolution": 1024, "y_resolution": 768})
print("Velocity contour plot generated and displayed.")

#### **Practice Questions:**

1.  Looking at the temperature plot, do the hot and cold fluids appear to be well-mixed by the time they reach the outlet? How can you tell?
2.  In the velocity plot, where is the velocity the highest? Why do you think this is the case? (Hint: Think about the shape of the pipe and conservation of mass).

### Step 5: Clean up

Finally, it's good practice to close the Fluent session to release the license and computer resources.

In [ ]:
# Close the Fluent session
session.exit()
print("Fluent session closed.")

--- 
## Conclusion & Further Exploration

Congratulations! You have successfully completed a full CFD simulation using ANSYS Fluent entirely from Python. You have:
*   Learned the fundamental physical principles behind CFD.
*   Walked through the standard CFD workflow: Preprocessing, Solving, and Post-processing.
*   Interactively simulated a mixing elbow problem.
*   Visualized and analyzed the results to understand fluid behavior.

You are VERY far ahead

#### **Ideas to Explore Next:**
*   **Change the BCs:** Go back to Step 3, change the inlet velocities or temperatures, and rerun the simulation to see how it affects the mixing.
*   **Change the Fluid:** The default fluid is air. You can try changing it to water (`setup.materials.database.copy_by_name(type="fluid", name="water-liquid")` and applying it to the fluid zone) to see how the properties of the fluid impact the result.
*   **Generate XY Plots:** Use Fluent's plotting tools to create a graph of temperature along the centerline of the pipe at the outlet.